After playing through with this notebook, please enter your thoughts in this Google Form: https://forms.gle/pMj3ZstRhUywx9cc7

# Input

In [1]:
user_prompt = "futuristic"
character_style = "lego"

In [2]:
from collections import namedtuple

Character = namedtuple('Character', [
    'complexion',
    'hair',
    'clothing',
    'accessories',
    'demeanor',
    'additional_details',
    'image_url'
])

# Part 1

In [3]:
!pip install --upgrade openai
!pip install transformers numpy torch opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64

In [4]:
from transformers import OwlViTProcessor, OwlViTForObjectDetection
from PIL import Image
import cv2
import numpy as np
import torch

processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
detector = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/613M [00:00<?, ?B/s]

In [12]:
import os
from getpass import getpass
from openai import OpenAI
if 'OPENAI_API_KEY' not in os.environ:
    print("You didn't set your OpenAI key to the OPENAI_API_KEY env var on the command line.")
    os.environ['OPENAI_API_KEY'] = getpass("Please enter your OpenAI API Key now: ")
client = OpenAI()

Please enter your OpenAI API Key now: ··········


In [6]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))

  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [13]:
import os
import requests
from PIL import Image, ImageDraw
import torch
from transformers import OwlViTProcessor, OwlViTForObjectDetection
import base64

os.makedirs('img_people', exist_ok=True)

processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

url = "photo.jpg"
image = Image.open(url)

texts = [["a photo of a person"]]

inputs = processor(text=texts, images=image, return_tensors="pt")

outputs = model(**inputs)

target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs=outputs, target_sizes=target_sizes, threshold=0.1)

descriptions = []

def generate_description(image_path):
    """Generate a detailed description of the person in the image using GPT-4."""

    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
            {
              "type": "text",
              "text": "Provide a detailed and comprehensive description of the person in the image, ensuring to cover all of the following aspects in a structured format: \n\n1. Complexion: Describe the skin tone and any noticeable features such as freckles, scars, or tattoos. \n2. Hair: Note the color, length, style, and any distinctive characteristics such as curls, bangs, or highlights. \n3. Clothing: Describe the type of clothing worn, including style, colors, and any logos or text. Mention the type of outfit (casual, formal, sporty, etc.). \n4. Accessories: List any visible accessories like glasses, jewelry, watches, or hats. \n5. Demeanor: Describe the person’s apparent mood or demeanor, based on their expression and posture. \n6. Additional Details: Include any other notable features such as piercings, makeup, or distinctive facial expressions. \n\nThis detailed description should capture every relevant aspect, ensuring reproducibility from the description alone."
            },
            {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
            },
          },
        ],
      }
    ],
    max_tokens=1000,
  )

    return response.choices[0].message.content

for i, result in enumerate(results[0]["boxes"]):
    xmin, ymin, xmax, ymax = map(int, result.tolist())
    person_image = image.crop((xmin, ymin, xmax, ymax))
    person_image_path = f'img_people/person_segment_{i}.png'
    person_image.save(person_image_path)

    description = generate_description(person_image_path)
    descriptions.append(description)


In [14]:
character_description=[]
def generate_extra_description(image_path): # this function is for character_desc in part3
    """Generate a detailed description of the person in the image using GPT-4."""
    result = []
    with open(image_path, "rb") as image_file:
        base64_image = base64.b64encode(image_file.read()).decode('utf-8')

    name_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
            {
              "type": "text",
              "text": "Give the person a name using his/her characteristics. Output only the name and remove any other words."
            },
            {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
            },
          },
        ],
      }
    ],
    max_tokens=1000,
  )
    personality_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
            {
              "type": "text",
              "text": "Describe the person's personality using his/her characteristics. Output only the personality and remove any other words."
            },
            {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
            },
          },
        ],
      }
    ],
    max_tokens=1000,
  )
    goal_response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
            {
              "type": "text",
              "text": "Describe the person's goal using his/her characteristics. Output only the goal and remove any other words."
            },
            {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
            },
          },
        ],
      }
    ],
    max_tokens=1000,
  )
    result.append(name_response.choices[0].message.content)
    result.append(personality_response.choices[0].message.content)
    result.append(goal_response.choices[0].message.content)
    return result

for i, result in enumerate(results[0]["boxes"]):
    person_image_path = f'img_people/person_segment_{i}.png'
    description = generate_extra_description(person_image_path)
    character_description.append(description)

In [15]:
character_desc=""
for index, desc in enumerate(character_description):
   character_desc += "Character "+str(index+1)+" - "+desc[0]+"\n"
   character_desc += "Personality: "+desc[1]+"\n"
   character_desc += "Goal:"+desc[2]+"\n"

In [16]:
cleaned_descriptions = []
characters = []
for element in descriptions:
  temp = ""
  temp_dict = {}
  splits = element.split("\n\n")
  for line in splits:
    if(line[0].isdigit()):
      temp += line
      line_split = line.split(":")
      if "complexion" in line_split[0].lower():
        temp_dict["complexion"]= line_split[1].strip()
      elif "hair" in line_split[0].lower():
        temp_dict["hair"]= line_split[1].strip()
      elif "clothing" in line_split[0].lower():
        temp_dict["clothing"]= line_split[1].strip()
      elif "accessories" in line_split[0].lower():
        temp_dict["accessories"]= line_split[1].strip()
      elif "demeanor" in line_split[0].lower():
        temp_dict["demeanor"]= line_split[1].strip()
      elif "additional details" in line_split[0].lower():
        temp_dict["additional_details"]= line_split[1].strip()
  cleaned_descriptions.append(temp)
  characters.append(temp_dict)

In [17]:
for index, desc in enumerate(cleaned_descriptions):
  response = client.images.generate(
    model="dall-e-3",
    prompt="Give me an image of a person in "+character_style+" style. The person has the following description:"+desc,
    size="1024x1024",
    quality="standard",
    n=1,
  )

  image_url = response.data[0].url
  characters[index]["image_url"] = image_url

# Output from Part 1

```characters: List[Character]```
## (images will be saved to runtime as well but image_url will be in Character named tuple)

# Part 2

input: user image, user text input

output: dictionary of locations, each location mapping to its own dictionary containing image URL (in runtime), description, connections, and challenges

In [18]:
from collections import defaultdict
from getpass import getpass
from copy import deepcopy
import PIL

import json
import os
import random

!pip install -e .'[dev]'
!pip install openai tiktoken
!pip install rembg
!pip install requests

Obtaining file:///content
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.5 MB/s eta 0:00:00


In [19]:
from openai import OpenAI

def get_features_in_image(image_url: str) -> str:
  client = OpenAI()
  with open(image_url, "rb") as image_file:
    base64_image = base64.b64encode(image_file.read()).decode('utf-8')
  response = client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": "Write a detailed description of this image and its setting."},
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
              "detail": "high"
            },
          },
        ],
      }
    ],
    max_tokens=300,
  )
  return response.choices[0].message.content

In [20]:
def get_locations_descriptions(user_prompt: str, features: str, sys_prompt: str) -> str:
    messages = [
      {'role': 'system', 'content': sys_prompt},
      {'role': 'user', 'content': "Game in the style of " + user_prompt + " given the following setting: " + features}
    ]

    client = OpenAI()

    response = client.chat.completions.create(
      model='gpt-4',
      messages=messages,
      temperature=1,
      max_tokens=2048,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    locations_descr = response.choices[0].message.content
    return locations_descr

In [21]:
def generate_location_descriptions(setting: str, user_text_input: str) -> list:
  sys_prompt_var = "You are creating a simple board game. Given the style of game that the user inputs along with a list of features that describe the setting of the game, create a description of a potential location within the larger setting of the game (that does not have to include the features but should be somewhat related) in paragraph form in 4 sentences."
  first_location_description = get_locations_descriptions(user_text_input, setting, sys_prompt_var)

  sys_prompt_var = "You are creating a simple board game. Given the style of game that the user inputs along with a list of features that describe the setting of the game, create a description of a potential location within the larger setting of the game (that does not have to include the features but should be somewhat related) in paragraph form in 4 sentences. Generate a location description that is ENTIRELY DIFFERENT FROM " + first_location_description
  second_location_description = get_locations_descriptions(user_text_input, setting, sys_prompt_var)

  sys_prompt_var = "You are creating a simple board game. Given the style of game that the user inputs along with a list of features that describe the setting of the game, create a description of a potential location within the larger setting of the game (that does not have to include the features but should be somewhat related) in paragraph form in 4 sentences. Generate a location description that is ENTIRELY DIFFERENT FROM " + first_location_description + "and NOT anything similar to" + second_location_description
  third_location_description = get_locations_descriptions(user_text_input, setting, sys_prompt_var)

  sys_prompt_var = "You are creating a simple board game. Given the style of game that the user inputs along with a list of features that describe the setting of the game, create a description of a potential location within the larger setting of the game (that does not have to include the features but should be somewhat related) in paragraph form in 4 sentences. Generate a location description that is ENTIRELY DIFFERENT FROM " + first_location_description + "and NOT anything similar to" + second_location_description + "and NOT anything similar to" + third_location_description
  fourth_location_description = get_locations_descriptions(user_text_input, setting, sys_prompt_var)

  return [first_location_description, second_location_description, third_location_description, fourth_location_description]

In [22]:
def get_locations_challenges(location_description: str) -> str:
    sys_prompt = "You are creating a simple board game with 4 locations. Given the description for one of these locations, create challenges involving dice rolls. Output the descriptions and rules of the challenges in paragraph form."
    messages = [
      {'role': 'system', 'content': sys_prompt},
      {'role': 'user', 'content': str(location_description)}
    ]

    client = OpenAI()

    response = client.chat.completions.create(
      model='gpt-4',
      messages=messages,
      temperature=1,
      max_tokens=2048,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    challenges_descr = response.choices[0].message.content
    return challenges_descr

In [23]:
def generate_challenge_descriptions(descr_location: list) -> list:
  first_challenge_description = get_locations_challenges(descr_location[0])
  second_challenge_description = get_locations_challenges(descr_location[1])
  third_challenge_description = get_locations_challenges(descr_location[2])
  fourth_challenge_description = get_locations_challenges(descr_location[3])

  return [first_challenge_description, second_challenge_description, third_challenge_description, fourth_challenge_description]

In [24]:
def get_locations_connections(description_one: str, description_two: str) -> str:
    sys_prompt = "You are creating a simple board game with 4 locations. Given the description for one location and the description for another, write a 4 sentence paragraph describing a connection between the two locations incorporating dice rolls for different possible transitions depending on the outcome through which the player can travel between them."
    messages = [
      {'role': 'system', 'content': sys_prompt},
      {'role': 'user', 'content': str("location 1: " + description_one + "; location 2: " + description_two)}
    ]

    client = OpenAI()

    response = client.chat.completions.create(
      model='gpt-4',
      messages=messages,
      temperature=1,
      max_tokens=2048,
      top_p=1.0,
      frequency_penalty=0,
      presence_penalty=0
    )

    connections_descr = response.choices[0].message.content
    return connections_descr

In [25]:
def generate_connections_descriptions(descr_location: list) -> list:
  # location 1: 2, 3, 4
  # location 2: 1, 3
  # location 3: 1, 2
  # location 4: 1
  one_and_two = get_locations_connections(descr_location[0], descr_location[1])
  one_and_three = get_locations_connections(descr_location[0], descr_location[2])
  one_and_four = get_locations_connections(descr_location[0], descr_location[3])
  three_and_two = get_locations_connections(descr_location[1], descr_location[2])
  return [one_and_two, one_and_three, one_and_four, one_and_two, three_and_two, one_and_three, three_and_two, one_and_four]

In [26]:
import PIL
from PIL import Image, ImageDraw, ImageOps
import rembg
from rembg import remove

def edit_images_to_tiles(image_url: str) -> str:
  img = Image.open(image_url)
  background = Image.new("RGBA", img.size, (0,0,0,0))

  mask = Image.new("RGBA", img.size, 0)
  draw = ImageDraw.Draw(mask)
  draw.regular_polygon((512,512,512), 6, rotation=360, fill='green', outline=None)

  composite_img = Image.composite(img, background, mask)
  hexagon_img = remove(composite_img)
  hexagon_img.save(image_url)

  # bordered_img = ImageOps.expand(hexagon_img, border=50, fill="white")
  # bordered_img.save(image_url)
  return image_url

In [27]:
import requests

def save_image_url_as_file(file_name: str, url: str) -> str:
  image_bits = requests.get(url).content
  with open(file_name, 'wb') as handler:
    handler.write(image_bits)
    return file_name

In [28]:
def get_location_images_from_descr(loc_description: str) -> str:
  # calls DALLE for each location description, outputs an image URL
  client = OpenAI()
  response = client.images.generate(
    model="dall-e-3",
    prompt="Create an image of a scene based on the location description: " + loc_description,
    size="1024x1024",
    quality="standard",
    n=1,
  )

  image_url = response.data[0].url
  return image_url

In [29]:
def generate_location_tiles(desc_locations: list) -> list:
  loc_image_1 = get_location_images_from_descr(desc_locations[0])
  new_url_1 = save_image_url_as_file('loc_1.png', loc_image_1)
  tile_1 = edit_images_to_tiles('/content/loc_1.png')

  loc_image_2 = get_location_images_from_descr(desc_locations[1])
  new_url_2 = save_image_url_as_file('loc_2.png', loc_image_2)
  tile_2 = edit_images_to_tiles('/content/loc_2.png')

  loc_image_3 = get_location_images_from_descr(desc_locations[2])
  new_url_3 = save_image_url_as_file('loc_3.png', loc_image_3)
  tile_3 = edit_images_to_tiles('/content/loc_3.png')

  loc_image_4 = get_location_images_from_descr(desc_locations[3])
  new_url_4 = save_image_url_as_file('loc_4.png', loc_image_4)
  tile_4 = edit_images_to_tiles('/content/loc_4.png')

  return [tile_1, tile_2, tile_3, tile_4]

In [30]:
new_image_path = f'/content/photo.png'
Image.open("/content/photo.jpg").save(new_image_path)

features = get_features_in_image("/content/photo.png")
descr_locations_list = generate_location_descriptions(features, user_prompt)

In [31]:
challenges_locations_list = generate_challenge_descriptions(descr_locations_list)
connections_locations_list = generate_connections_descriptions(descr_locations_list)

In [32]:
location_image_url_list = generate_location_tiles(descr_locations_list)

100%|███████████████████████████████████████| 176M/176M [00:00<00:00, 47.6GB/s]


In [33]:
locations_info_dictionary = {}

locations_info_dictionary["loc1"] = {}
locations_info_dictionary["loc2"] = {}
locations_info_dictionary["loc3"] = {}
locations_info_dictionary["loc4"] = {}

locations_info_dictionary["loc1"]["description"] = descr_locations_list[0]
locations_info_dictionary["loc2"]["description"] = descr_locations_list[1]
locations_info_dictionary["loc3"]["description"] = descr_locations_list[2]
locations_info_dictionary["loc4"]["description"] = descr_locations_list[3]

locations_info_dictionary["loc1"]["connections"] = [connections_locations_list[0], connections_locations_list[1], connections_locations_list[2]]
locations_info_dictionary["loc2"]["connections"] = [connections_locations_list[3], connections_locations_list[4]]
locations_info_dictionary["loc3"]["connections"] = [connections_locations_list[5], connections_locations_list[6]]
locations_info_dictionary["loc4"]["connections"] = [connections_locations_list[7]]

locations_info_dictionary["loc1"]["challenges"] = challenges_locations_list[0]
locations_info_dictionary["loc2"]["challenges"] = challenges_locations_list[1]
locations_info_dictionary["loc3"]["challenges"] = challenges_locations_list[2]
locations_info_dictionary["loc4"]["challenges"] = challenges_locations_list[3]

images_for_locations = {}

images_for_locations["loc1"] = location_image_url_list[0]
images_for_locations["loc2"] = location_image_url_list[1]
images_for_locations["loc3"] = location_image_url_list[2]
images_for_locations["loc4"] = location_image_url_list[3]

map_location_info = str(locations_info_dictionary)

# Part 3

In [34]:
# user_input = "Take this photo of my friends on a boat, and turn it into a futuristic adventure game!"

map_desc = map_location_info

scene_desc = locations_info_dictionary["loc1"]["description"]

# characters_desc = """Character A - The Navigator (Person sitting at the boat's helm)
# Personality: Calm and collected, with an analytical mind.
# Goals: Seeking to chart and explore uncharted territories in the galaxy, uncovering ancient alien civilizations and technologies.
# Character B - The Daredevil (Person in mid-jump with arms wide open)
# Personality: Thrill-seeker, always looking for the next adrenaline rush.
# Goals: Known to take on the most dangerous missions, aiming to earn fame and prove their unmatched bravery.
# Character C - The Visionary (Person standing on the boat, preparing to jump)
# Personality: Charismatic leader, dreaming of a better future for humanity.
# Goals: To unite the scattered human colonies under a single banner and to find a new home planet.
# Character D - The Engineer (Person standing on the boat, facing away)
# Personality: Inventive and resourceful, a problem-solver at heart.
# Goals: Designing a revolutionary spaceship propulsion system and establishing the first human settlement on a distant world.
# Character E - The Biologist (Person in the water, taking a picture)
# Personality: Inquisitive and nurturing, a lover of all living things.
# Goals: Discovering new life forms and ensuring the preservation of biodiversity across the cosmos."""

In [35]:
import json

# User Input Prompt: user_prompt
# User Input Image: img_url
# Characters List: characters
#   - Character: 'complexion','hair','clothing','accessories','demeanor','additional_details','image_url'
# Map Locations Information: map_location_info
# Map Locations Images: images_for_locations

output_format = """
{
  "GameTitle": "name of the game",
  "GameDescription": "description of the game"
  "Characters": [
    {
      "Name": the name of the character/role,
      "Appearance": a description of what the character looks like and their demeanor
      "Description": a description of the character/role's actions and goals are, and any additional details to note
    }
  ],
  "GameplayMechanics": "description of the gameplay mechanics -- should explain how the game flows and what players do each round"
  "Map & Challenges": [
    {
      "LocationName": name of the location,
      "Description": a description of the location
      "Challenge": the challenge associated with the location that the players need to overcome
      "Connections": list of names of connected locations; e.g. "Location A, Location B, Location C"
    }
  ],
  "WinningTheGame": "define the end/win state of the game; what state is fulfilled for the game to end"
}
"""

prompt = "Write the rules and description for a one page role-playing game. The game you generate (and the challenges and missions) should be specific so that the game is playable without a game master. Make sure to specify in detail some blocks/conflicts/challengers users can play through. The mechanics can be simple as long as the key conflict is specified. Your output should take the format of a JSON file that lists each of the following sections: GameTitle, Game Description, Characters (Name, Skills, Goal), Gameplay Mechanics, Map & Challenges (Location Name, Challenge, Outcome), Key Locations, and Winning the Game. Be sure to include all those sections, and take special care to format your response exactly as this example file, with no text prior to this format: " + output_format + "\n Base the game on the style, characters, scene, and map as follows. The user wants a particular style -- they said: \"" + user_prompt + "\" The characters are as follows: \"" + str(characters) + "\" For the characters section, make sure to maintain the order of the characters in your output JSON so it matches the order that they are listed as here. The map is as follows: \"" + str(map_location_info) + "\" and again you should maintain order of the map locations in your JSON output."
response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": prompt},
      ],
    }
  ],
  max_tokens=2000,
)

rules_guide = response.choices[0].message.content
print(rules_guide)


{
  "GameTitle": "Echoes of Tomorrow",
  "GameDescription": "In this futuristic role-playing game, players journey through distant worlds and high-tech habitats, encountering complex challenges and vibrant locales. Navigating through these locations, players aim to master their environments, solve technological hitches, and adapt to both urban and space-bound settings. The goal is to efficiently manage resources and overcome spatial challenges to secure a prosperous and advanced lifestyle across galaxies.",
  "Characters": [
    {
      "Name": "Tech Navigator",
      "Appearance": "Black, short, straight hair with slight volume, light to medium smooth skin tone, black crew neck t-shirt, rectangular thin-framed glasses, neutral expression, relaxed but attentive demeanor.",
      "Description": "A skilled problem solver and tech-savvy individual, the Tech Navigator's primary goal is to successfully interact with and manipulate futuristic technologies to enhance living conditions across 

In [36]:
!pip install reportlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.7 MB/s eta 0:00:00


In [37]:
import requests

from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image
from reportlab.lib.styles import getSampleStyleSheet

data = json.loads(rules_guide)
# print(data)

pdf_file = "rpg_ruleset.pdf"
doc = SimpleDocTemplate(pdf_file, pagesize=letter)

# Creating the stylesheet
styles = getSampleStyleSheet()
title_style = styles["Title"]
subtitle_style = styles["Heading2"]
content_style = styles["Normal"]

content = []

# Title
content.append(Paragraph(data["GameTitle"], title_style))
content.append(Paragraph(data["GameDescription"], content_style))
content.append(Paragraph("\n", content_style))

# Characters
count = 0
content.append(Paragraph("Characters", subtitle_style))
for character in data["Characters"]:
    content.append(Paragraph(f"Name: {character['Name']}", subtitle_style))
    content.append(Paragraph(f"Appearance: {character['Appearance']}", content_style))
    content.append(Paragraph(f"Description: {character['Description']}", content_style))
    content.append(Paragraph("\n", content_style))

# Character Portraits
for c in characters:
  img_url = c["image_url"]
  count += 1
  new_img_url = save_image_url_as_file(str(count) + 'char_img.jpg', img_url)
  image = Image(new_img_url, width=200, height=200) # drawing on the pdf canvas
  content.append(image)
  content.append(Paragraph("\n", content_style))


# Map & Challenges
content.append(Paragraph("Map & Challenges", subtitle_style))
for location in data["Map & Challenges"]:
    content.append(Paragraph(f"Location: {location['LocationName']}", subtitle_style))
    content.append(Paragraph(f"Description: {location['Description']}", content_style))
    content.append(Paragraph(f"Challenge: {location['Challenge']}", content_style))
    content.append(Paragraph(f"Connections: {location['Connections']}", content_style))
    content.append(Paragraph("\n", content_style))

# Map Location Images
for loc in images_for_locations:
  img_url = images_for_locations[loc]
  image = Image(img_url, width=200, height=200) # drawing on the pdf canvas
  content.append(image)
  content.append(Paragraph("\n", content_style))

# Gameplay Mechanics
content.append(Paragraph("Gameplay Mechanics", subtitle_style))
content.append(Paragraph(data["GameplayMechanics"], content_style))
content.append(Paragraph("\n", content_style))

# Winning The Game
content.append(Paragraph("Winning The Game", subtitle_style))
content.append(Paragraph(data["WinningTheGame"], content_style))

# Output the PDF file
doc.build(content)